# MCS MEA — Pair Viewer Quick Start

HERE ARE THE GUI RELEVANT CHANGES

This notebook gives you three reliable ways to launch the CTZ–VEH 2×2 GUI (raw + IFR) from Jupyter.
- Inline (Jupyter-native, recommended)
- Subprocess via absolute script path (no import issues)
- Module mode via `-m scripts.pair_viewer` (requires repo root as cwd)


In [ ]:
# Ensure repo root is importable (so mcs_mea_analysis is found)
import sys
from pathlib import Path
def _ensure_repo_on_path():
    here = Path.cwd()
    for cand in [here, *here.parents]:
        if (cand / 'mcs_mea_analysis').exists():
            if str(cand) not in sys.path:
                sys.path.insert(0, str(cand))
            return cand
    return here
REPO_ROOT = _ensure_repo_on_path(); REPO_ROOT

In [ ]:
# Build readiness and ready pairs (self-contained)
import pandas as pd
from pathlib import Path as _Path
from mcs_mea_analysis.ready import ReadinessConfig, build_ready_index
from mcs_mea_analysis.pairings import PairingIndex
OUTPUT_ROOT = _Path('/Volumes/Manny2TB/mcs_mea_outputs') if (_Path('/Volumes/Manny2TB/mcs_mea_outputs').exists()) else _Path('_mcs_mea_outputs_local')
ready_csv, _, ready_rows = build_ready_index(ReadinessConfig(output_root=OUTPUT_ROOT, require_ifr_npz=True))
df = pd.DataFrame(ready_rows)
px = PairingIndex.from_ready_rows(df.to_dict('records'), group_by_round=True)
pairs_df = pd.DataFrame(px.pairs_dataframe())
ready_pairs_df = pairs_df.query('pair_status=="ready_pair"').copy()
print('Ready pairs:', len(ready_pairs_df))
ready_pairs_df.head(10)

## Quick Start Launcher (pick mode, plate, pair, channel)

In [ ]:
# Helper functions and 3 launch methods
import sys as _sys, subprocess, shlex
from pathlib import Path as _P
import pandas as pd

def _repo_root(start) -> _P:
    p = _P(start).resolve()
    while p != p.parent:
        if (p / 'scripts' / 'pair_viewer.py').exists():
            return p
        p = p.parent
    return _P.cwd()

def _get_pair_context(plate:int, pair_idx:int=0):
    rpp = ready_pairs_df[ready_pairs_df['plate']==plate].reset_index(drop=True)
    if rpp.empty: raise RuntimeError(f'No ready pairs for plate {plate}')
    if not (0 <= pair_idx < len(rpp)): raise RuntimeError(f'pair_idx {pair_idx} out of range [0..{len(rpp)-1}]')
    row = rpp.iloc[pair_idx]
    ctz_h5 = df.loc[df['recording_stem']==row['ctz_stem'], 'path']
    veh_h5 = df.loc[df['recording_stem']==row['veh_stem'], 'path']
    chem_c = df.loc[df['recording_stem']==row['ctz_stem'], 'chem_timestamp']
    chem_v = df.loc[df['recording_stem']==row['veh_stem'], 'chem_timestamp']
    return row, ctz_h5, veh_h5, chem_c, chem_v

def open_pair_inline(plate:int, pair_idx:int=0, ch:int=0):
    ip = get_ipython()
    if ip is not None:
        ip.run_line_magic('gui', 'qt5')
    from mcs_mea_analysis.pair_viewer_gui import PairInputs, launch_pair_viewer
    row, ctz_h5, veh_h5, chem_c, chem_v = _get_pair_context(plate, pair_idx)
    pin = PairInputs(
        plate=int(row['plate']) if pd.notna(row['plate']) else None,
        round=str(row['round']) if pd.notna(row['round']) else None,
        ctz_npz=_P(row['ctz_npz_path']),
        veh_npz=_P(row['veh_npz_path']),
        ctz_h5=_P(ctz_h5.iloc[0]) if not ctz_h5.empty else None,
        veh_h5=_P(veh_h5.iloc[0]) if not veh_h5.empty else None,
        chem_ctz_s=float(chem_c.iloc[0]) if not chem_c.empty and pd.notna(chem_c.iloc[0]) else None,
        chem_veh_s=float(chem_v.iloc[0]) if not chem_v.empty and pd.notna(chem_v.iloc[0]) else None,
        initial_channel=int(ch),
    )
    launch_pair_viewer(pin)

def open_pair_subprocess_abs(plate:int, pair_idx:int=0, ch:int=0):
    ROOT = _repo_root(_P.cwd())
    SCRIPT = ROOT / 'scripts' / 'pair_viewer.py'
    row, ctz_h5, veh_h5, chem_c, chem_v = _get_pair_context(plate, pair_idx)
    cmd = [ _sys.executable, str(SCRIPT),
            '--ctz-npz', str(row['ctz_npz_path']), '--veh-npz', str(row['veh_npz_path']),
            '--plate', str(row['plate']), '--round', str(row['round']), '--ch', str(int(ch)) ]
    if not ctz_h5.empty: cmd += ['--ctz-h5', str(ctz_h5.iloc[0])]
    if not veh_h5.empty: cmd += ['--veh-h5', str(veh_h5.iloc[0])]
    if not chem_c.empty and pd.notna(chem_c.iloc[0]): cmd += ['--chem-ctz', str(float(chem_c.iloc[0]))]
    if not chem_v.empty and pd.notna(chem_v.iloc[0]): cmd += ['--chem-veh', str(float(chem_v.iloc[0]))]
    print('Launching (abs):', ' '.join(shlex.quote(str(c)) for c in cmd))
    subprocess.Popen(cmd, cwd=str(ROOT))

def open_pair_module_mode(plate:int, pair_idx:int=0, ch:int=0):
    ROOT = _repo_root(_P.cwd())
    row, ctz_h5, veh_h5, chem_c, chem_v = _get_pair_context(plate, pair_idx)
    cmd = [ _sys.executable, '-m', 'scripts.pair_viewer',
            '--ctz-npz', str(row['ctz_npz_path']), '--veh-npz', str(row['veh_npz_path']),
            '--plate', str(row['plate']), '--round', str(row['round']), '--ch', str(int(ch)) ]
    if not ctz_h5.empty: cmd += ['--ctz-h5', str(ctz_h5.iloc[0])]
    if not veh_h5.empty: cmd += ['--veh-h5', str(veh_h5.iloc[0])]
    if not chem_c.empty and pd.notna(chem_c.iloc[0]): cmd += ['--chem-ctz', str(float(chem_c.iloc[0]))]
    if not chem_v.empty and pd.notna(chem_v.iloc[0]): cmd += ['--chem-veh', str(float(chem_v.iloc[0]))]
    print('Launching (-m):', ' '.join(shlex.quote(str(c)) for c in cmd))
    subprocess.Popen(cmd, cwd=str(ROOT))


In [ ]:
# ipywidgets launcher
try:
    import ipywidgets as W
    plates = sorted([int(p) for p in ready_pairs_df['plate'].dropna().unique().tolist()])
    dd_mode = W.ToggleButtons(options=[('inline','inline'), ('subprocess(abs)','abs'), ('module -m','mod')], description='Mode:')
    dd_plate = W.Dropdown(options=plates, description='Plate:')
    dd_pair = W.Dropdown(options=[], description='Pair:')
    sl_ch = W.IntSlider(value=0, min=0, max=59, step=1, description='Channel:')
    btn = W.Button(description='Launch GUI', button_style='primary')
    out = W.Output()
    def _upd(*_):
        rpp = ready_pairs_df[ready_pairs_df['plate']==dd_plate.value].reset_index(drop=True)
        dd_pair.options = [(f"{i}: {rpp.loc[i,'ctz_stem']} vs {rpp.loc[i,'veh_stem']}", i) for i in range(len(rpp))]
        sl_ch.value = 0
    dd_plate.observe(_upd, names='value')
    if plates: dd_plate.value = plates[0]
    def _go(_):
        row_idx = int(dd_pair.value) if dd_pair.value is not None else 0
        if dd_mode.value == 'inline':
            open_pair_inline(int(dd_plate.value), row_idx, int(sl_ch.value))
        elif dd_mode.value == 'abs':
            open_pair_subprocess_abs(int(dd_plate.value), row_idx, int(sl_ch.value))
        else:
            open_pair_module_mode(int(dd_plate.value), row_idx, int(sl_ch.value))
    btn.on_click(_go)
    display(W.VBox([W.HBox([dd_mode, dd_plate, dd_pair, sl_ch, btn]), out]))
except Exception as e:
    print('ipywidgets unavailable:', e)
